In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StringType, BinaryType, IntegerType, DoubleType, TimestampType, DateType
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
from pyspark.storagelevel import StorageLevel
from typing import Union, Optional

# --- Credenciais AWS ---
accessKeyId = ""
secretAccessKey = ""

# --- Sessão Spark ---
def create_spark_session() -> SparkSession:
    spark = (
        SparkSession
        .builder
        .appName("Gold Zone")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
        .getOrCreate()
    )
    
    spark.sparkContext.setLogLevel("WARN")

    conf = spark.sparkContext._jsc.hadoopConfiguration()
    conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
    conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
    conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    conf.set("fs.s3a.fast.upload", "true")
    conf.set("fs.s3a.bucket.all.committer.magic.enabled", "true")
    conf.set("fs.s3a.directory.marker.retention", "keep")
    conf.set("spark.driver.extraClassPath", "/usr/local/spark/jars/*")
    conf.set("spark.driver.memory", "8g")
    conf.set("spark.executor.memory", "16g")
    conf.set("fs.s3a.access.key", accessKeyId)
    conf.set("fs.s3a.secret.key", secretAccessKey)

    return spark

In [2]:
def create_tempview_from_delta(spark, delta_table_path, view_name):
    """
    Cria uma tempView a partir de uma tabela Delta.
    
    Parâmetros:
    - delta_table_path (str): Caminho para a tabela Delta
    - view_name (str): Nome da tempView a ser criada
    
    Retorno:
    - None (cria a tempView no SparkSession atual)
    """
   
    if spark is None:
        raise ValueError("Nenhuma SparkSession ativa encontrada")
    
    spark.read.format("delta").load(delta_table_path).createOrReplaceTempView(view_name)
    
    print(f"TempView '{view_name}' criada com sucesso a partir da tabela Delta em: {delta_table_path}")

In [3]:
%%time
spark = create_spark_session()

tabelas_silver = [
    "dim_cliente",
    "dim_profissional",
    "dim_servico",
    "dim_tempo",
    "fato_agendamento",
    "fato_pagamento",
    "dim_promocao"
]

silver = "s3a://dev-lab-02-us-east-2-silver/"
gold   = "s3a://dev-lab-02-us-east-2-gold/"

CPU times: user 7.63 ms, sys: 16.1 ms, total: 23.8 ms
Wall time: 2.47 s


In [4]:
for t in tabelas_silver:
    create_tempview_from_delta(spark, f"{silver}{t}",t )

TempView 'dim_cliente' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-silver/dim_cliente
TempView 'dim_profissional' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-silver/dim_profissional
TempView 'dim_servico' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-silver/dim_servico
TempView 'dim_tempo' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-silver/dim_tempo
TempView 'fato_agendamento' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-silver/fato_agendamento
TempView 'fato_pagamento' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-silver/fato_pagamento
TempView 'dim_promocao' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-silver/dim_promocao


In [5]:
for t in tabelas_silver:
    print(t)
    spark.sql(f"select * from {t} limit 1").printSchema()

dim_cliente
root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- primeiro_nome: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- email: string (nullable = true)
 |-- data_nascimento: date (nullable = true)
 |-- faixa_etaria: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- dia_semana_nascimento: string (nullable = true)
 |-- estacao_nascimento: string (nullable = true)
 |-- data_cadastro: date (nullable = true)
 |-- dias_como_cliente: integer (nullable = true)
 |-- ativo: boolean (nullable = true)
 |-- segmento_tempo: string (nullable = true)
 |-- origem_dados: string (nullable = true)
 |-- data_carga: timestamp (nullable = true)
 |-- data_exclusao: timestamp (nullable = true)

dim_profissional
root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- email: string (nullable = true)
 |-- especialidade: string (nullable = true)
 |-- data_admissao: date (nullabl

## 📘 `faturamento_mensal`

### 🧩 Descrição da tabela:

Tabela que consolida o faturamento mensal da empresa, permitindo análises por forma de pagamento, serviço e especialidade.

### 📐 Regra de negócio:

Considera todos os pagamentos ativos com data válida e agrupa por mês e ano da data de pagamento.

### 🧾 Dicionário de dados:

| Campo                  | Tipo          | Descrição                                                     |
| ---------------------- | ------------- | ------------------------------------------------------------- |
| `ano`                  | integer       | Ano da data de pagamento.                                     |
| `mes`                  | integer       | Mês da data de pagamento.                                     |
| `forma_pagamento`      | string        | Forma de pagamento utilizada (ex: crédito, débito, dinheiro). |
| `nome_servico`         | string        | Nome do serviço prestado.                                     |
| `qtd_pagamentos`       | bigint        | Quantidade de pagamentos realizados no período.               |
| `valor_total_faturado` | decimal(10,2) | Soma dos valores pagos no período.                            |
| `ticket_medio`         | decimal(10,2) | Média dos valores pagos no período.                           |

---

In [32]:
df_faturamento_mensal = spark.sql(
"""
SELECT
    YEAR(data_pagamento) AS ano,
    MONTH(data_pagamento) AS mes,
    forma_pagamento,
    nome_servico,
    COUNT(*) AS qtd_pagamentos,
    SUM(valor_total) AS valor_total_faturado,
    ROUND(AVG(valor_total), 2) AS ticket_medio
FROM fato_pagamento
WHERE ativo = TRUE AND data_pagamento IS NOT NULL
and data_pagamento_date is not null
GROUP BY
    YEAR(data_pagamento),
    MONTH(data_pagamento),
    forma_pagamento,
    nome_servico

"""
)
df_faturamento_mensal.show()

+----+---+---------------+--------------------+--------------+--------------------+------------+
| ano|mes|forma_pagamento|        nome_servico|qtd_pagamentos|valor_total_faturado|ticket_medio|
+----+---+---------------+--------------------+--------------+--------------------+------------+
|2019|  3|            pix|  Hidratação Capilar|             8|              526.00|       65.75|
|2019|  6|  cartao_debito|  Hidratação Capilar|             4|              331.00|       82.75|
|2020|  2|            pix|   Barba com Navalha|            11|              650.00|       59.09|
|2020| 10|            pix|       Corte Moicano|            12|              698.50|       58.21|
|2020| 11|       dinheiro|       Corte Moicano|             7|              350.00|       50.00|
|2021|  7|            pix|      Corte Infantil|             5|              295.50|       59.10|
|2021|  8|  cartao_debito|      Barba Completa|             9|              555.00|       61.67|
|2021| 11|       dinheiro|    

## 📘 `desempenho_profissionais`

### 🧩 Descrição da tabela:

Consolida o desempenho financeiro e operacional dos profissionais com base nos atendimentos e pagamentos realizados.

### 📐 Regra de negócio:

Baseada em agendamentos e pagamentos ativos. Considera apenas agendamentos com pagamentos associados.

### 🧾 Dicionário de dados:

| Campo                  | Tipo          | Descrição                                                      |
| ---------------------- | ------------- | -------------------------------------------------------------- |
| `id_profissional`      | integer       | Identificador do profissional.                                 |
| `nome_profissional`    | string        | Nome completo do profissional.                                 |
| `qtd_atendimentos`     | bigint        | Número total de agendamentos realizados.                       |
| `valor_total_recebido` | decimal(10,2) | Soma dos valores pagos em atendimentos do profissional.        |
| `ticket_medio`         | decimal(10,2) | Valor médio por atendimento.                                   |
| `dias_ativos_mes`      | integer       | Dias distintos com pelo menos um atendimento realizado no mês. |


In [31]:
df_desempenho_profissionais = spark.sql(
"""
SELECT
    profissional_id AS id_profissional,
    p.nome_profissional,
    YEAR(p.data_pagamento_date) AS ano,
    MONTH(p.data_pagamento_date) AS mes,
    COUNT(*) AS qtd_atendimentos,
    SUM(valor_total) AS valor_total_recebido,
    ROUND(AVG(valor_total), 2) AS ticket_medio,
    COUNT(DISTINCT DATE(p.data_pagamento_date)) AS dias_ativos_mes
FROM fato_agendamento AS f
JOIN fato_pagamento AS p
  ON f.id = p.agendamento_id
WHERE f.ativo = TRUE AND p.ativo = TRUE
and data_pagamento_date is not null
GROUP BY
    profissional_id,
    p.nome_profissional,
    YEAR(p.data_pagamento_date),
    MONTH(p.data_pagamento_date)

"""
)
df_desempenho_profissionais.show()

+---------------+-----------------+----+---+----------------+--------------------+------------+---------------+
|id_profissional|nome_profissional| ano|mes|qtd_atendimentos|valor_total_recebido|ticket_medio|dias_ativos_mes|
+---------------+-----------------+----+---+----------------+--------------------+------------+---------------+
|              9|    Roberto Nunes|2024| 10|              16|             1195.00|       74.69|             10|
|              7|     Lucas Mendes|2024|  9|              12|              890.00|       74.17|             10|
|              7|  Antônio Barbosa|2024| 11|              18|             1239.00|       68.83|             11|
|              7|    Marcos Santos|2022|  9|              51|             3434.50|       67.34|             20|
|              6| Ricardo Ferreira|2023|  7|              15|             1086.50|       72.43|             11|
|              6|  Antônio Barbosa|2024|  3|              11|              924.00|       84.00|         

## 📘 `agendamentos_diarios`

### 🧩 Descrição da tabela:

Monitora os agendamentos realizados diariamente, incluindo cancelamentos e métricas operacionais básicas.

### 📐 Regra de negócio:

Considera apenas agendamentos ativos, agrupando por data e dia da semana.

### 🧾 Dicionário de dados:

| Campo                         | Tipo    | Descrição                                                                        |
| ----------------------------- | ------- | -------------------------------------------------------------------------------- |
| `data_agendamento`            | date    | Data do agendamento.                                                             |
| `qtd_agendamentos`            | bigint  | Número total de agendamentos no dia.                                             |
| `qtd_cancelamentos`           | bigint  | Quantidade de agendamentos cancelados no dia.                                    |
| `qtd_novos_clientes`          | bigint  | Quantidade de clientes que agendaram pela primeira vez (0 dias de antecedência). |
| `taxa_ocupacao_profissionais` | double  | Proporção média de uso dos profissionais no dia (%).                             |
| `dia_semana`                  | string  | Nome do dia da semana (ex: Segunda-feira).                                       |
| `fim_de_semana`               | boolean | Indica se o dia é sábado ou domingo.                                             |

In [33]:
df_agendamentos_diarios = spark.sql(
"""
SELECT
    DATE(data_hora) AS data_agendamento,
    COUNT(*) AS qtd_agendamentos,
    SUM(CASE WHEN status = 'cancelado' THEN 1 ELSE 0 END) AS qtd_cancelamentos,
    SUM(CASE WHEN dias_antecedencia = 0 THEN 1 ELSE 0 END) AS qtd_novos_clientes,
    ROUND(100.0 * COUNT(*) / COUNT(DISTINCT profissional_id), 2) AS taxa_ocupacao_profissionais,
    dia_semana_nome AS dia_semana,
    MAX(CASE WHEN dia_semana_nome IN ('Sábado', 'Domingo') THEN TRUE ELSE FALSE END) AS fim_de_semana
FROM fato_agendamento
WHERE ativo = TRUE
GROUP BY DATE(data_hora), dia_semana_nome
"""
)
df_agendamentos_diarios.show()

+----------------+----------------+-----------------+------------------+---------------------------+----------+-------------+
|data_agendamento|qtd_agendamentos|qtd_cancelamentos|qtd_novos_clientes|taxa_ocupacao_profissionais|dia_semana|fim_de_semana|
+----------------+----------------+-----------------+------------------+---------------------------+----------+-------------+
|      2021-04-23|              44|                3|                 0|                    1100.00|    Friday|        false|
|      2022-12-08|              48|                2|                 0|                    1200.00|  Thursday|        false|
|      2022-12-31|              47|                1|                 0|                    1175.00|  Saturday|        false|
|      2024-09-27|             108|                5|                 0|                    1080.00|    Friday|        false|
|      2021-12-29|              47|                2|                 0|                    1175.00| Wednesday|       

In [34]:
df_agendamentos_diarios.write.mode("overwrite").format("delta").save(f"{gold}agendamentos_diarios")
df_desempenho_profissionais.write.mode("overwrite").format("delta").save(f"{gold}desempenho_profissionais")
df_faturamento_mensal.write.mode("overwrite").format("delta").save(f"{gold}faturamento_mensal")